In [ ]:
# Load Dataset
train_data = open('train.csv').readlines()
test_data = open('test.csv').readlines()

# Separate header and data
train_header = train_data[0].strip().split(',')
train_data = [line.strip().split(',') for line in train_data[1:]]